In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from metloom.pointdata import SnotelPointData
import geopandas as gpd

In [25]:
start_date = datetime(2020, 12, 1)
end_date = datetime(2023, 3, 31)

# Pull desired variables from snotel to dataframe
snotel_point = SnotelPointData('1082:WY:SNTL', "MyStation")
df = snotel_point.get_hourly_data(
    start_date, end_date,
    [snotel_point.ALLOWED_VARIABLES.PRECIPITATIONACCUM, snotel_point.ALLOWED_VARIABLES.TEMP]
)

# Clean up the dataframe
df.reset_index(inplace=True)

# Rename columns
replace = {'ACCUMULATED PRECIPITATION':'accppt','AIR TEMP':'airtemp', 'datetime':'time'}
df.rename(columns=replace, inplace=True)
df.set_index('time', inplace=True)
# conver airtemp to C
df['airtempC'] = (df['airtemp'] - 32) * 5/9

# calc precip rate
df['precip'] = df['accppt'].diff()

In [26]:
df

,site,geometry,accppt,ACCUMULATED PRECIPITATION_units,airtemp,AIR TEMP_units,datasource,airtempC,precip
time,,,,,,,,,
2020-12-01 08:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),7.4,in,11.48,degF,NRCS,-11.4,NaN
2020-12-01 09:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),7.2,in,11.30,degF,NRCS,-11.5,-0.2
2020-12-01 10:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),7.2,in,11.30,degF,NRCS,-11.5,0.0
2020-12-01 11:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),7.2,in,10.40,degF,NRCS,-12.0,0.0
2020-12-01 12:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),7.2,in,10.58,degF,NRCS,-11.9,0.0
...,...,...,...,...,...,...,...,...,...
2023-03-31 04:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),22.8,in,14.72,degF,NRCS,-9.6,0.0
2023-03-31 05:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),22.8,in,14.18,degF,NRCS,-9.9,0.0
2023-03-31 06:00:00+00:00,1082:WY:SNTL,POINT Z (-110.92783 43.77933 9260.00000),22.8,in,14.18,degF,NRCS,-9.9,0.0


In [29]:
# Function to filter rows between December 1 and March 31
def is_winter_month(date):
    return (date.month == 12 or date.month == 1 or date.month == 2 or date.month == 3)

# Filter the DataFrame
winter_df = df[df.index.map(is_winter_month)]

# Calculate total snow and rain for the winter months with different thresholds
def calculate_snow_rain_fractions(df, threshold):
    snow = df.loc[df['airtempC'] < threshold, 'precip'].sum()
    rain = df.loc[df['airtempC'] >= threshold, 'precip'].sum()
    snow_frac = snow / (snow + rain) * 100 if (snow + rain) > 0 else 0
    return snow, rain, snow_frac

# Thresholds
thresholds = [0, -2, -4]

# Calculate and print results for each threshold
for threshold in thresholds:
    snow, rain, snow_frac = calculate_snow_rain_fractions(winter_df, threshold)
    print(f"Threshold: {threshold}°C")
    print(f"Total snow: {snow}")
    print(f"Total rain: {rain}")
    print(f"Fraction of precip that is snow: {snow_frac:.2f}%\n")


Threshold: 0°C
Total snow: 48.0
Total rain: 4.7
Fraction of precip that is snow: 91.08%

Threshold: -2°C
Total snow: 45.900000000000006
Total rain: 6.8
Fraction of precip that is snow: 87.10%

Threshold: -4°C
Total snow: 37.19999999999998
Total rain: 15.500000000000018
Fraction of precip that is snow: 70.59%

